# Jupyter Notebook to make CSV of the annotated recordings

Create a CSV file with the annotated information of the recordings from the Arab-Andalusian dataset.  
(Work in progress)  
The poetry form is not filled yet

In [81]:
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import sys
import os
import csv
from compmusic import dunya
sys.path.append('src/')
import model

apipath = 'api/andalusian/recording/'
dunya.set_token('<SET_YOUR_TOKEN_HERE>')

In [3]:
mbids=['a451a7fc-c53f-462a-b3fc-4377bb588105','8842c1f0-e261-4069-bd59-768bb9a3315c', 'f7bcb9af-6abb-4192-ae3d-37fa811034ce', 'b11237b9-d45b-4b3a-a97b-ab7d198f927f']
def extract_orchestra_name(info):
    return info['archive_url'][27:].split('_')[0]
def extract_mizan_name(info):
    return info['sections'][1]['mizan']['transliterated_name']

In [6]:
with open('arab_andalusian_verses.csv', 'w') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
        filewriter.writerow(['mbid', 'verse_name','poetry_form','t_s','t_e','score_s','score_e','mizan','orchestra','lyric'])
        for mbid in mbids:
            info = dunya.conn._dunya_query_json(apipath + mbid)
            # Compute type of poetry form each sanaa
            fn1 = os.path.join('sanaa_lyrics/transliterated/tsv',mbid+'.tsv')
            poetry_form = model.extract_poetry(fn1)

            #Compute verses onsets from Textgrid file
            fn2 = os.path.join('Textgrid', mbid + '.TextGrid') # Set path where the textgrid file is
            data_matrix = model.extract_data_matrix(fn2)

            #Compute verses onsets from score file
            fn3 = os.path.join('score_segmented', mbid + '-segmented.musicxml')
            verses_filt = model.extract_verses_score(fn3)

            # Compare the  lyric list and the verse list and check if they have same length, in a case some 
            # of them has one extra section
            matrix_for_csv=[]
            for d in data_matrix:
                for v in verses_filt:
                    if d[0] == v[0]:
                        matrix_for_csv.append([d[0],d[1], d[2], d[3], v[1],v[2]])

            # Write the data in CSV:
            for x in matrix_for_csv:
                filewriter.writerow([mbid,x[0],' ',x[2],x[3],x[4],x[5],extract_mizan_name(info), extract_orchestra_name(info), x[1]])
            
            print(mbid+': ')
            for key in poetry_form:
                print (key,':',poetry_form[key])
            print('\n')
    
   

a451a7fc-c53f-462a-b3fc-4377bb588105: 
Qasida : [1]
Moaxaja : [2, 3]
Zejel : [0, 4, 5, 6, 7, 8, 9, 10]
Barwala : []


8842c1f0-e261-4069-bd59-768bb9a3315c: 
Qasida : [0]
Moaxaja : []
Zejel : [2, 4, 5, 6, 7, 8, 9, 10]
Barwala : [1, 3]


f7bcb9af-6abb-4192-ae3d-37fa811034ce: 
Qasida : [0, 2, 10]
Moaxaja : [3, 13]
Zejel : [4, 5, 6, 7, 8, 9, 11, 12, 14]
Barwala : [1]


b11237b9-d45b-4b3a-a97b-ab7d198f927f: 
Qasida : [0, 1, 4, 12]
Moaxaja : [3, 13]
Zejel : [2, 5, 7, 8, 9, 10, 11, 14]
Barwala : []


